In [ ]:
%config Completer.use_jedi = False

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import re
import ast
import matplotlib.pyplot as plt
import time
import geojson
from geojson import Feature, FeatureCollection, Point

In [ ]:
export_path = # папка для выгрузки geojson

In [ ]:
HOST = 'https://www.bustime.ru/'
URL = 'https://www.bustime.ru/nizhniy-novgorod/transport/2021-01-18/'
HEADERS = {'accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
           'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36'}

In [ ]:
def get_html(url, params=''):
    req = requests.get(url, headers=HEADERS, params=params)
    return req

In [ ]:
html = get_html(URL)

In [ ]:
# html.text

In [ ]:
""" Забираем id маршрутов (зачем?)""" 

def get_id_list(html):
    soup = BeautifulSoup(html)
    route_list = {}
    for option in soup.find_all('option'):
        route_list[option['value']]=option.text
    id_list = list(route_list.keys())
    return id_list

In [ ]:
id_list = get_id_list(html.text)

In [ ]:
""" Достаём из html страницы все ссылки """

hrefs = []
soup = BeautifulSoup(html.text)
for a in soup.find_all('a'):
    hrefs.append(a['href'])

In [ ]:
# hrefs

In [ ]:
""" Из списка с ссылками достаём только страницы маршрутов """
""" Переписать в функциональном виде """

hrefs_sort = []
for i in range(len(hrefs)):
    if hrefs[i].find('/nizhniy-novgorod/transport/2021-01-18/') == 0:
        hrefs_sort.append(hrefs[i])
    else:
        continue
invalid_indices = []
for i in range(len(hrefs_sort)):
    if hrefs_sort[i].find('page') >= 0:
        invalid_indices.append(i)
    else:
        continue
del hrefs_sort[invalid_indices[0]:invalid_indices[-1]+1]

In [ ]:
# invalid_indices

In [ ]:
hrefs_sort

In [ ]:
""" Создаём список с ссылками на страницы """

pg = [*range(1,30)]
page_urls = []
for i in pg:
    page_urls.append(URL + 'page-' + str(i))

In [ ]:
page_urls

In [ ]:
ROUTE_URL = HOST + hrefs_sort[0]
route_html = get_html(ROUTE_URL)

In [ ]:
# route_html.text

In [ ]:
soup = BeautifulSoup(route_html.text, 'html.parser')
script = soup.find_all('script')
# script

In [ ]:
var = re.compile(r"var (.*);",re.DOTALL)
result = re.findall(var,str(script))
result_list = str(result).split(';')
track_line = result_list[3]

temp_new = track_line.split('\\n')
temp_new = ''.join(temp_new)
_replace = {'\\':' ',
           'var track = ':'',
           "'":'"',
           'uniqueid':'"uniqueid"',
           'gosnum':'"gosnum"',
           'bortnum':'"bortnum"',
           'timestamp':'"timestamp"',
           'bus_id':'"bus_id"',
           'heading':'"heading"',
           'speed':'"speed"',
           'lon':'"lon"',
           'lat':'"lat"',
           'direction':'"direction"',
           '[':'',
           ']':'',
           'parseFloat( ':'',
           '.replace( ", ", ". ") )':'',
           ' ':'',
           '},{':'}, {'}

In [ ]:
def mult_replace(text, dict_):
    for i,j in dict_.items():
        text = text.replace(i, j)
    return text
routes_list = mult_replace(temp_new, _replace).split(', ')
routes_list[-1] = routes_list[-1].rstrip(',')

In [ ]:
def geojsonize(_list, path, uid):
    features=[]
    for i in range(len(_list)):
        temp_json = json.loads(_list[i])
        geometry = [float(temp_json["lon"].replace(',','.')),
                    float(temp_json["lat"].replace(',','.'))]
        props = {"uid":temp_json["uniqueid"],
                 "gosnum":temp_json["gosnum"],
                 "bortnum":temp_json["bortnum"],
                 "timestamp":temp_json["timestamp"],
                 "bus_id":temp_json["bus_id"],
                 "heading":temp_json["heading"],
                 "speed":temp_json["speed"],
                 "direction":temp_json["direction"]}
        features.append(Feature(geometry=geojson.Point((float(temp_json["lon"].replace(',','.')), 
                                                        float(temp_json["lat"].replace(',','.')))),
                                properties=props))
    feature_collection = FeatureCollection(features)
    with open(path+'points_'+uid+'.geojson', 'w',encoding='utf8') as f:
        geojson.dump(feature_collection, f,ensure_ascii=False)
    print('200')

In [ ]:
hrefs_ = []
for i in range(len(page_urls)):
    PAGE_URL = page_urls[i]
    html = get_html(PAGE_URL)
    soup = BeautifulSoup(html.text)
    for a in soup.find_all('a'):
        hrefs_.append(a['href'])
    print(i, 'done')
print('complete')

In [ ]:
len(hrefs_)

In [ ]:
hrefs_

In [ ]:
hrefs_sort_ = []
for i in range(len(hrefs_)):
    if hrefs_[i].find('/nizhniy-novgorod/transport/2021-01-18/') == 0:
        hrefs_sort_.append(hrefs_[i])
    else:
        continue

# СДЕЛАТЬ КОМБИНАЦИЮ С НИЖНИМ ПРАВИЛОМ ЧЕРЕЗ AND 

In [ ]:
invalid_indices_ = []
for i in range(len(hrefs_sort_)):
    if hrefs_sort_[i].find('page') >= 0:
        invalid_indices_.append(i)
    else:
        continue

In [ ]:
invalid_hrefs = []
for i in range(len(invalid_indices_)):
    invalid_hrefs.append(hrefs_sort_[invalid_indices_[i]])

In [ ]:
hrefs_sort_npg = set(hrefs_sort_) - set(invalid_hrefs)
hrefs_sort_npg = list(hrefs_sort_npg)
len(hrefs_sort_npg)

In [ ]:
cnt = 0
fail = 0

for i in range(len(hrefs_sort_npg)):
    
    # скачиваем html страницы автобуса
    route_id = hrefs_sort_npg[i].strip('/nizhniy-novgorod/transport/2021-01-18/')
    print(route_id+':')
    ROUTE_URL = HOST + hrefs_sort_npg[i]
    route_html = get_html(ROUTE_URL)
    print('\thtml автобуса',route_id,'скачан')
    
    # ищем скрипт на странице
    soup = BeautifulSoup(route_html.text, 'html.parser')
    script = soup.find_all('script')
    print('\tcкрипт автобуса',route_id,'найден')
    
    # вытаскиваем блок с json
    var = re.compile(r"var (.*);",re.DOTALL)
    result = re.findall(var,str(script))
    result_list = str(result).split(';')
    track_line = result_list[3]
    temp_new = track_line.split('\\n')
    temp_new = ''.join(temp_new)
    print('\tjson автобуса',route_id,'скачан')
    
    # убираем весь мусор
    routes_list = mult_replace(temp_new, _replace).split(', ')
    routes_list[-1] = routes_list[-1].rstrip(',')
    
    # сохраняем в geojson
    print('\tМаршрут',route_id+'. Статус:')
    cnt += 1
    try:
        geojsonize(routes_list, export_path, route_id)
        print('\n',str(round(cnt/len(hrefs_sort_npg)*100,1))+'%')
        print('\n')
        time.sleep(1)
    except:
        print('\t\t\t\tFail')
        fail += 1
        continue

In [ ]:
fail